**PRE PROCESSING PHASE**

In this phase we need to generate the final dataset (Image1,Image2,Caption). 

The steps to follow are this:
1) Generate the embendings of the images with BLIP
2) Compute the similarity in orther to obtain the pair (Image1,Image2) with a fine grained way.
3) Use a captioning model for the generation of the two caption (Caption1,Caption2).
4) Use an LLM for the creation of the "Caption of Difference".
5) Concatenate all the things for the final creation of the dataset (Image1,Image2,Caption)

In [6]:
#import libraries
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
import os
from tqdm import tqdm
import lovely_tensors as lt
import csv
import torch
from tqdm import tqdm

In [7]:
# Load the model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [13]:
#estrarre la path delle immagini da dentro la cartella imagesf2
images_dir = "D:\\COMPUTER VISION EXAM\\ComputeVisionProject\\images-hd\\"
image_paths = []

for root, dirs, files in os.walk(images_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.jpeg'):
            image_path = os.path.join(root, file)
            image_paths.append(image_path)


len(image_paths)

#creare un file csv con tutte le path
with open('C:\\Users\\Utente\\Desktop\\PROJECTS\\Image_Difference_Captioning\\data\\processed\\image_paths.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image_path"])
    for image_path in image_paths:
        writer.writerow([image_path])

In [26]:
#save the image path column into a list, skip the first element

image_path_list = []
with open('C:\\Users\\Utente\\Desktop\\PROJECTS\\Image_Difference_Captioning\\data\\processed\\image_paths.csv', mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        image_path_list.append(lines[0])

image_path_list = image_path_list[1:]

In [28]:
batch_size = 32
image_features = {}
import torch
from tqdm import tqdm
from PIL import Image

# Assicurati che il modello sia sulla GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

with torch.no_grad():
    for i in tqdm(range(0, len(image_path_list), batch_size)):
        batch_paths = image_path_list[i:i + batch_size]
        batch = [Image.open(path) for path in batch_paths]
        inputs = processor(images=batch, return_tensors="pt", padding=True)
        
        # Trasferisci gli input sulla GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        features = model.get_image_features(**inputs)
        image_features.update({path: feature for path, feature in zip(batch_paths, features)})

  0%|          | 0/3640 [00:00<?, ?it/s]c:\Users\Utente\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\clip\modeling_clip.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 3640/3640 [39:44<00:00,  1.53it/s] 


In [31]:
from safetensors.torch import save_file, load_file
save_file(image_features,"C:\\Users\\Utente\\Desktop\\PROJECTS\\Image_Difference_Captioning\\data\\processed\\image_features.safetensors")


In [33]:
image_features = load_file("C:\\Users\\Utente\\Desktop\\PROJECTS\\Image_Difference_Captioning\\data\\processed\\image_features.safetensors")
image_features_tensor = torch.stack(list(image_features.values()))
with open('C:\\Users\\Utente\\Desktop\\PROJECTS\\Image_Difference_Captioning\\data\\processed\\similarities_Pairs.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Image 1', 'Image 2', 'Similarity'])
    # compute similarity between each vector and the entire tensor
    for i in tqdm(range(image_features_tensor.shape[0])):
        similarity = torch.nn.functional.cosine_similarity(image_features_tensor[i], image_features_tensor)
        # save in the csv file only the pair of images with the largest similarity for each image image_features_tensor[i]
        similarity[i] = 0
        max_similarity = torch.max(similarity)
        max_index = torch.argmax(similarity)
        writer.writerow([image_path_list[i], image_path_list[max_index], max_similarity.item()])
        

  0%|          | 98/116475 [00:06<1:59:15, 16.26it/s]


KeyboardInterrupt: 